<br><br><br><br><br>

# More stuff!

(Based on requests from yesterday.)

<br><br><br><br><br>

### NumExpr

For "flat" expressions, like ufuncs, but can do many operations in one pass.

<center><img src="img/flat.png" width="25%"></center>

NumExpr compiles a limited set of expressions (mathematical formulae, no loops) for its own virtual machine. (Being a much simpler virtual machine than Python's, it can be evaluated more quickly.)

<br>

If you have Pandas installed, you have NumExpr. Pandas uses NumExpr when `apply` is given an expression in a string.

<br>

**Note:** I didn't find any documentation that says you can turn NumExpr expressions into ufuncs. (Shocking!) The author is working on NumExpr 3: I'll suggest it.

In [ ]:
import numpy, numexpr

a = numpy.random.uniform(5, 10, 1000000)
b = numpy.random.uniform(10, 20, 1000000)
c = numpy.random.uniform(-0.1, 0.1, 1000000)

roots1 = (-b + numpy.sqrt(b**2 - 4*a*c)) / (2*a)

roots2 = numexpr.evaluate("(-b + sqrt(b**2 - 4*a*c)) / (2*a)")   # how does it know a, b, c?

roots2 = numexpr.evaluate("(-b + sqrt(b**2 - 4*a*c)) / (2*a)", global_dict={"a": a, "b": b, "c": c})

roots2 = numexpr.evaluate("(-b + sqrt(b**2 - 4*a*c)) / (2*a)", global_dict=globals())

print((roots1 == roots2).all())

In [ ]:
%%timeit

roots1 = (-b + numpy.sqrt(b**2 - 4*a*c)) / (2*a)

In [ ]:
%%timeit

roots2 = numexpr.evaluate("(-b + sqrt(b**2 - 4*a*c)) / (2*a)")

In [ ]:
import numba

@numba.jit(nopython=True)
def roots_numba(a, b, c):
    out = numpy.empty_like(a)
    for i in range(len(a)):
        out[i] = (-b[i] + numpy.sqrt(b[i]**2 - 4*a[i]*c[i])) / (2*a[i])
    return out

roots3 = roots_numba(a, b, c)

print(numpy.allclose(roots1, roots3))

In [ ]:
%%timeit

roots3 = roots_numba(a, b, c)

In [ ]:
# Maybe fastmath=True will help? (The `-ffast-math` flag helps a lot in C/C++.)

@numba.jit(nopython=True, fastmath=True)
def roots_numba(a, b, c):
    out = numpy.empty_like(a)
    for i in range(len(a)):
        out[i] = (-b[i] + numpy.sqrt(b[i]**2 - 4*a[i]*c[i])) / (2*a[i])
    return out

roots3 = roots_numba(a, b, c)

print(numpy.allclose(roots1, roots3))

In [ ]:
%%timeit

roots3 = roots_numba(a, b, c)

<br><br><br>

So NumExpr even beats Numba (in this example, which is in NumExpr's sweet spot).

   * **When to use NumExpr:** you have a complex formula composed entirely of ufuncs—all flat operations. If you've already done `from numpy import *` (bad practice), then all you need to do is surround your formula with quotes and wrap it with `numexpr.evaluate()`. Otherwise, you have to drop `numpy.` or `np.` from the beginning of each function name.
   * **When to use Numba:** you have a more complex function to calculate than something strictly one-to-one, or your work is likely to grow that way.

_Anyway,_ both are very easy to add and remove. Do whatever is easiest until it's not.

<br><br><br>

<br><br><br><br><br>

# Making your own ufunc

<br><br><br><br><br>

<br><br><br>

### Directly in C?

I took a look at the [documentation on creating a Numpy ufunc in C](https://docs.scipy.org/doc/numpy/user/c-info.ufunc-tutorial.html), which is the standard way, and concluded that it's not for the faint of heart. If you're primarily working in C/C++, this would be the most natural way to expose your program's functionality as a Numpy ufunc, but it gets into C details I'd rather not do here.

Also, it doesn't seem to be possible in pybind11. (At least, I've found no documentation for it.)

<br><br><br>

In [ ]:
# With Python: the Numpy library has a "vectorize" function, but it's not useful for speed.
# It just runs your Python function with a ufunc-like interface.

import time, math

quadratic_slow = numpy.vectorize(lambda a, b, c: (-b + math.sqrt(b**2 - 4*a*c)) / (2*a))

a = numpy.random.uniform(5, 10, 10000000)
b = numpy.random.uniform(10, 20, 10000000)
c = numpy.random.uniform(-0.1, 0.1, 10000000)

starttime = time.time()
quadratic_slow(a, b, c)
print("Python time:", time.time() - starttime)

starttime = time.time()
(-b + numpy.sqrt(b**2 - 4*a*c)) / (2*a)
print("Numpy time:", time.time() - starttime)

In [ ]:
# With Numba: this was a motivating case for Numba, to create compiled ufuncs.

import numba

@numba.vectorize
def quadratic(a, b, c):
    return (-b + math.sqrt(b**2 - 4*a*c)) / (2*a)    # a, b, c are scalars; output is scalar

a = numpy.random.uniform(5, 10, 1000000)
b = numpy.random.uniform(10, 20, 1000000)
c = numpy.random.uniform(-0.1, 0.1, 1000000)

print(quadratic(a, b, c))                             # a, b, c are arrays; output is an array

In [ ]:
import awkward

# Now that it's a ufunc, the awkward library says, "I know what quadratic is!"

a = awkward.fromiter([[ 5.9,  5.5,  9.7], [], [ 7.7,  5.6]])
b = awkward.fromiter([[17.6, 14.0, 19.5], [], [15.4, 12.8]])
c = awkward.fromiter([[ 0.8, -0.9,  1.0], [], [-1.7,  0.2]])

print(quadratic(a, b, c))

In [ ]:
import dask.array

# Dask library says, "I know what quadratic is!"

a = dask.array.from_array(numpy.random.uniform(5, 10, 1000000), chunks=100000)
b = dask.array.from_array(numpy.random.uniform(10, 20, 1000000), chunks=100000)
c = dask.array.from_array(numpy.random.uniform(-0.1, 0.1, 1000000), chunks=100000)

quadratic(a, b, c)

In [ ]:
((-b + numpy.sqrt(b**2 - 4*a*c)) / (2*a)).visualize()
# quadratic(a, b, c).visualize()

<br><br><br><br>

This could be a good way to use Dask: wrap up your most complex logic in ufuncs to simplify the task graph.

<br>

(It should also work with Dask distributed, if the remote machine has Numba to recompile the function when it receives it, though I haven't tried that.)

<br><br><br><br>

In [ ]:
import cupy

# CuPy tries sending it to Numba, but Numba doesn't recognize the CuPy array type.

a = cupy.asarray(numpy.random.uniform(5, 10, 1000000))
b = cupy.asarray(numpy.random.uniform(10, 20, 1000000))
c = cupy.asarray(numpy.random.uniform(-0.1, 0.1, 1000000))

quadratic(a, b, c)

In [ ]:
import pandas

# Pandas says, "I know what quadratic is!"

df = pandas.DataFrame(dict(
    a = numpy.random.uniform(5, 10, 1000000),
    b = numpy.random.uniform(10, 20, 1000000),
    c = numpy.random.uniform(-0.1, 0.1, 1000000)))

df["roots"] = quadratic(df["a"], df["b"], df["c"])
df

<br><br><br><br><br>

In database-speak, applying a ufunc to columns of a Pandas table is like using a UDF (user-defined function) or "stored procedure" in SQL.

<br><br><br><br><br>

<br><br><br><br><br>

# Dask for out-of-memory analytics

<br><br><br><br><br>

In [ ]:
import dask.dataframe

exoplanets = dask.dataframe.read_csv("data/nasa-exoplanets.csv")
exoplanets

In [ ]:
# We can restructure the table without reading anything from disk.

df = exoplanets[["ra", "dec", "st_dist", "st_mass", "st_rad", "pl_orbsmax", "pl_orbeccen", "pl_orbper", "pl_bmassj", "pl_radj"]]
df.columns = pandas.MultiIndex.from_arrays([["star"] * 5 + ["planet"] * 5,
    ["right asc. (deg)", "declination (deg)", "distance (pc)", "mass (solar)", "radius (solar)", "orbit (AU)", "eccen.", "period (days)", "mass (Jupiter)", "radius (Jupiter)"]])
df

In [ ]:
density = df[("planet", "mass (Jupiter)")] / df[("planet", "radius (Jupiter)")]
density

In [ ]:
density.visualize()

In [ ]:
density.compute()

<br><br><br><br><br>

It works, but the functionality is more limited than Pandas. If you're considering an analysis using Dask DataFrames, _start_ the analysis with Dask DataFrames, so that you don't find out that you depend on something after you've written all of your analysis code.

<br><br><br><br><br>